# データシステムの知能化とデザイン
# 第3回課題
## 学籍番号：62116674
## 氏名：藤本巧実

### 課題１

データの準備

In [1]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)
#使用カラムを限定する
usecols=['Rooms', 'Type', 'Price', 'Method', 'SellerG',
        'Distance', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Propertycount']
if not os.path.exists('Melbourne_housing_FULL.csv'):
    !wget "https://drive.google.com/uc?export=download&id=1Ow53Wl7g40Pr1ExUoJ4P-IrSejpzUalt" -O Melbourne_housing_FULL.csv

df=pd.read_csv("Melbourne_housing_FULL.csv",usecols=usecols)

--2023-10-24 14:09:01--  https://drive.google.com/uc?export=download&id=1Ow53Wl7g40Pr1ExUoJ4P-IrSejpzUalt
Resolving drive.google.com (drive.google.com)... 108.177.13.139, 108.177.13.102, 108.177.13.138, ...
Connecting to drive.google.com (drive.google.com)|108.177.13.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ijden6ctsjjivv6d6viajilpfsqn2ucq/1698156525000/14553810797225214496/*/1Ow53Wl7g40Pr1ExUoJ4P-IrSejpzUalt?e=download&uuid=d2c861a2-e044-4245-af4b-9aa5a19ca525 [following]
--2023-10-24 14:09:03--  https://doc-00-7k-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ijden6ctsjjivv6d6viajilpfsqn2ucq/1698156525000/14553810797225214496/*/1Ow53Wl7g40Pr1ExUoJ4P-IrSejpzUalt?e=download&uuid=d2c861a2-e044-4245-af4b-9aa5a19ca525
Resolving doc-00-7k-docs.googleusercontent.com (doc-00-7k-docs.googleusercontent.com)... 173.194.218.132, 2607

Pandasによる内容確認

In [2]:
df

,Rooms,Type,Price,Method,SellerG,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Propertycount
0,2,h,NaN,SS,Jellis,2.5,2.0,1.0,1.0,126.0,NaN,NaN,4019.0
1,2,h,1480000.0,S,Biggin,2.5,2.0,1.0,1.0,202.0,NaN,NaN,4019.0
2,2,h,1035000.0,S,Biggin,2.5,2.0,1.0,0.0,156.0,79.0,1900.0,4019.0
3,3,u,NaN,VB,Rounds,2.5,3.0,2.0,1.0,0.0,NaN,NaN,4019.0
4,3,h,1465000.0,SP,Biggin,2.5,3.0,2.0,0.0,134.0,150.0,1900.0,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,4,h,1480000.0,PI,Jas,6.3,4.0,1.0,3.0,593.0,NaN,NaN,6543.0
34853,2,h,888000.0,SP,Sweeney,6.3,2.0,2.0,1.0,98.0,104.0,2018.0,6543.0
34854,2,t,705000.0,S,Jas,6.3,2.0,1.0,2.0,220.0,120.0,2000.0,6543.0
34855,3,h,1140000.0,SP,hockingstuart,6.3,NaN,NaN,NaN,NaN,NaN,NaN,6543.0


データの確認

In [3]:
df.dropna(subset=['Price'], inplace=True) #inplaceでdfそのものを修正する
print("全データ数")
print(len(df["Rooms"])) # Roomsに欠損がないことを知っているので
print("\n欠損数の確認")
print(df.isnull().sum())
#カテゴリ値を数値変換
le = preprocessing.LabelEncoder()
df[[ 'Type', 'Method', 'SellerG']]=df[[ 'Type', 'Method', 'SellerG']].apply(le.fit_transform)
df_train=df.drop("Price",axis=1)
df_label=df["Price"].values
print("\n訓練データの確認")
print(df_train)
print("\n訓練ラベルの確認(つまりお部屋のお値段、これを当てる)")
print(df_label)

全データ数
27247

欠損数の確認
Rooms                0
Type                 0
Price                0
Method               0
SellerG              0
Distance             1
Bedroom2          6441
Bathroom          6447
Car               6824
Landsize          9265
BuildingArea     16591
YearBuilt        15163
Propertycount        3
dtype: int64

訓練データの確認
       Rooms  Type  Method  SellerG  Distance  Bedroom2  Bathroom  Car  \
1          2     0       1       32       2.5       2.0       1.0  1.0   
2          2     0       1       32       2.5       2.0       1.0  0.0   
4          3     0       3       32       2.5       3.0       2.0  0.0   
5          3     0       0       32       2.5       3.0       2.0  1.0   
6          4     0       4      206       2.5       3.0       1.0  2.0   
...      ...   ...     ...      ...       ...       ...       ...  ...   
34852      4     0       0      140       6.3       4.0       1.0  3.0   
34853      2     0       3      290       6.3       2.0       2.0 

### 課題2

データの準備

In [4]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import copy
def run_model(df_train,df_label):
    params={
        'objective': 'regression',
        'random_state' : 1,
        "metric": "rmse",
        }
    kfold = 5
    score=0
    kf = KFold(n_splits=kfold,shuffle=True,random_state=94) #交差検証する関数を定義、データをkfold個にわけて1つをテスト用、残りを訓練に使う
    importance=0
    for i, (train_index, test_index) in enumerate(kf.split(df_train, df_label)): #定義したKFoldでdf_trainとdf_labelを分割
        print('[Fold %d/%d]' % (i + 1, kfold))
        X_train, X_valid = df_train.iloc[train_index], df_train.iloc[test_index] #train_indexなどは「選択した要素の配列」0から要素数あり、kfoldの確率で抜けている
        y_train, y_valid = df_label[train_index], df_label[test_index] #上と同様だが、要素が一つしかないのでこれでよい
        dtrain = lgb.Dataset(X_train, label=y_train) #まとめてデータセットを構成する
        dvalid = lgb.Dataset(X_valid, label=y_valid)
        bst = lgb.train(params, dtrain, num_boost_round=1000,valid_sets=[dtrain, dvalid])
        # LightGBMで学習、1000回試行して、ベストなものを返す
        importance += pd.DataFrame(bst.feature_importance(), index=df_train.columns, columns=['importance'])
        score+=bst.best_score["valid_1"]['rmse']
    return importance,score/5

欠損値を含むデータの学習

In [5]:
imp,score = run_model(df_train,df_label)
print(score)
print(imp)
scored = score

[Fold 1/5]
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.111983 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1284
[LightGBM] [Info] Number of data points in the train set: 21797, number of used features: 12
[LightGBM] [Info] Start training from score 1042830.453962
[Fold 2/5]
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000816 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1286
[LightGBM] [Info] Number of data points in the train set: 21797, number of used features: 12
[LightGBM] [Info] Start training from score 1050089.052255
[Fold 3/5]
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000831 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can

pandasで補完

In [6]:
#pandasで補完
dfp = df_train.interpolate(limit_direction='both')
df_train_fp=pd.DataFrame(dfp)
df_train_fp.columns=df_train.columns
imp,score=run_model(df_train_fp,df_label)
print(score)
scorep = score

[Fold 1/5]
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000782 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1853
[LightGBM] [Info] Number of data points in the train set: 21797, number of used features: 12
[LightGBM] [Info] Start training from score 1042830.453962
[Fold 2/5]
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000781 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1852
[LightGBM] [Info] Number of data points in the train set: 21797, number of used features: 12
[LightGBM] [Info] Start training from score 1050089.052255
[Fold 3/5]
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000771 seconds.
You can set `force_row_wise=tr

結果(RMSE)の比較

In [8]:
print("結果")
print("何もしない", scored)
print("Pandasで補完", scorep)

結果
何もしない 300266.36862001906
Pandasで補完 318582.4338602943


補完しない方がデータの性能が良い（？）→イレギュラー